In [6]:
import numpy as np
import pandas as pd
import bs4
from bs4 import BeautifulSoup
import requests

In [3]:
with open('../nasa_api.txt', 'r') as file:
    apikey = file.read()
# context manager 'with', 'r' means read

# Headers/parameters
headers = {
    "apikey": apikey
}

#### Astronomy Picture of the Day

In [7]:
# API endpoint for Astronomy Picture of the Day (APOD)
url = f'https://api.nasa.gov/planetary/apod?api_key={apikey}'

# Make a GET request to fetch the APOD data
response = requests.get(url)

if response.status_code == 200:
    apod_data = response.json()
    print("Title:", apod_data['title'])
    print("Date:", apod_data['date'])
    print("Explanation:", apod_data['explanation'])
    print("URL:", apod_data['url'])
else:
    print("Failed to fetch data:", response.status_code)

Title: Energetic Particle Strikes the Earth
Date: 2023-12-05
Explanation: It was one of the most energetic particles ever known to strike the Earth -- but where did it come from?  Dubbed Amaterasu after the Shinto sun goddess, this particle, as do all cosmic rays that strike the Earth's atmosphere, caused an air shower of electrons, protons, and other elementary particles to spray down onto the Earth below. In the featured illustration, a cosmic ray air shower is pictured striking the Telescope Array in Utah, USA, which recorded the Amaterasu event in 2021 May.  Cosmic ray air showers are common enough that you likely have been in a particle spray yourself, although you likely wouldn't have noticed.  The origin of this energetic particle, likely the nucleus of an atom, remains a mystery in two ways. First, it is not known how any single particle or atomic nucleus can practically acquire so much energy, and second, attempts to trace the particle back to where it originated did not indic

#### Mars Rover Photo

In [10]:
url = f'https://api.nasa.gov/mars-photos/api/v1/rovers/curiosity/photos?sol=1000&api_key={apikey}'

response = requests.get(url)

if response.status_code == 200:
    rover_data = response.json()
    photos = rover_data['photos']
    if photos:
        photo = photos[0]  # Fetching the first photo (index 0)
        print("Image:", photo['img_src'])
        print("Earth Date:", photo['earth_date'])
    else:
        print("No photos found for the given criteria.")
else:
    print("Failed to fetch data:", response.status_code)


Image: http://mars.jpl.nasa.gov/msl-raw-images/proj/msl/redops/ods/surface/sol/01000/opgs/edr/fcam/FLB_486265257EDR_F0481570FHAZ00323M_.JPG
Earth Date: 2015-05-30
